## Config
- Board model
<img src="./img/5.jpg" alt="drawing" style="width:200px;"/>

```C++
const int N = 4;

vector<int> board;
void printResult() {
    for(int i=0; i<N; ++i) {
        for(int j=0; j<N; ++j) {
            if(i == board[j])
                cout << 1 << ' ';
            else
                cout << 0 << ' ';
        }
        cout << endl;
    }
    cout << endl;
}
```
- Cost function
<img src="./img/6.jpg" alt="drawing" style="width:200px;"/>

```C++
int getCost(const vector<int> &board) {
    int cost = 0;

    // Check all rows and cols
    for(int i=0; i<N-1; ++i) {
        for(int j=i+1; j<N; ++j) {
            // same row
            if(board[i] == board[j])
                ++cost;

            // same diag
            int offset = j - i;
            if(board[i] == board[j] - offset || board[i] == board[j] + offset)
                ++cost;
        }
    }
    return cost;
}
```

### Hill climbing
- Each move = pick 1 Queen at 1 col -> move to a new row -> evaluate new cost
- Search all possible move -> choose the move with the lowest cost
- Prone to Exact Algorithm

<img src="./img/7.jpg" alt="drawing" style="width:200px;"/>


```C++
void solve() {
    // Init board
    //     {1,1,1,1},
    //     {0,0,0,0},
    //     {0,0,0,0},
    //     {0,0,0,0},
    board.assign(N, 0);

    // Loop until board cost = 0
    int best_cost = getCost(board);
    while(best_cost > 0) {
        // Search all possible moves -> find the lowest cost possible
        map<pair<int,int>, int> moves;
        for(int c=0; c<N; ++c) for(int r=0; r<N; ++r) {
            // Skip current possition
            if(board[c] == r)
                continue;

            // Pick 1 queen -> move to new rc to evaluate
            vector<int> board_copy(board);
            board_copy[c] = r;
            int new_cost = getCost(board_copy);

            // Update board best_cost
            if(new_cost <= best_cost)
                moves[{r,c}] = best_cost = new_cost;
        }

        // List all possible best move
        vector<pair<int,int>> best_moves;
        for(auto it=moves.begin(); it!=moves.end(); ++it)
            if(it->second == best_cost)
                best_moves.push_back(it->first);

        // Pick a random best move -> Update board
        pair<int, int> row_col = best_moves[randInt(0,best_moves.size())];
        board[row_col.second] = row_col.first;
    }
    
    // Print result
    printResult();
}
```
- Convergence
<img src="./img/8.png" alt="drawing" style="width:200px;"/>


### Stimulated Annealing
- Probability of acceptance p
    - $p = min(1,e^{\frac{\Delta E}{T}})$
        + $\Delta E = cur\_cost - new\_cost$ 
        + T = temperature

- An example of T function
    + $T(t+1) = max(T(t)*\alpha, 0.01)$
    + $\alpha$ = 0.95
- Prone to Approx. Algorithm

```C++
vector<int> nextStage(const vector<int> &board, int cur_cost, int T) {
    // Init a copied board
    vector<int> board_copy(board);
    bool acceptance = false;

    while(!acceptance) {
        // Select a random move
        int r = randInt(0, N);
        int c = randInt(0, N);
        if(board_copy[c] == r)
            continue;

        board_copy[c] = r;
        int new_cost = getCost(board_copy);

        // if new_cost is better -> accept this stage
        if(new_cost < cur_cost)
            acceptance = true;

        // If new_cost is not better -> setup a probability to accept this stage
        else {
            int delta = cur_cost - new_cost;
            double prob = min(1.0, exp(static_cast<double>(delta) / T));
            acceptance = randTrueFalse(prob);
        }
    }

    return board_copy;
}


void solve() {
    // Init board
    //     {1,1,1,1},
    //     {0,0,0,0},
    //     {0,0,0,0},
    //     {0,0,0,0},
    board.assign(N, 0);

    // Config params
    int T = N*N;
    double alpha = 0.8;
    int time_= 0, timeout = 1000;

    // Find next Stage
    int cost = getCost(board);
    while(cost > 0 && time_++ < timeout) {
        // Get board next stage
        cost = getCost(board);
        board = nextStage(board, cost, T);

        // Update T
        T = max(T*alpha, 0.01);
    }

    // Print result
    printResult();
}
```

- Convergence
<img src="./img/9.png" alt="drawing" style="width:200px;"/>